In [4]:
from cnocr import CnOcr

img_fp = './sample_image.jpg'
# check available models on https://github.com/breezedeus/CnOCR/
ocr = CnOcr(det_model_name='ch_PP-OCRv4_det_server',rec_model_name='ch_PP-OCRv4_server')  
out = ocr.ocr(img_fp)

print(out)
# for the first time running the script, it will download the onnx of models from url
# you can specify rec_root and det_root, to determin where the onnx will be saved/ was saved
# Default `~/.cnocr`, that create location like `~/.cnocr/2.3/densenet_lite_136-gru`

[{'text': '中华民國114年1-2月份', 'score': 0.9176208, 'position': array([[1048,  462],
       [1569,  452],
       [1570,  505],
       [1049,  515]], dtype=int32)}, {'text': '收銀機統一發票', 'score': 0.9112493, 'position': array([[1045,  524],
       [1410,  518],
       [1411,  582],
       [1046,  588]], dtype=int32)}, {'text': '收執联', 'score': 0.7539992, 'position': array([[1131,  590],
       [1406,  588],
       [1407,  652],
       [1131,  654]], dtype=int32)}, {'text': 'HW22497914', 'score': 0.99384403, 'position': array([[ 935,  696],
       [1521,  694],
       [1521,  798],
       [ 935,  800]], dtype=int32)}, {'text': '中民国统一', 'score': 0.81089485, 'position': array([[1174,  860],
       [1410,  854],
       [1411,  891],
       [1175,  897]], dtype=int32)}, {'text': 'RIPUBL', 'score': 0.59527034, 'position': array([[1217,  901],
       [1284,  901],
       [1284,  915],
       [1217,  915]], dtype=int32)}, {'text': '大坪林三号小吃店', 'score': 0.99753845, 'position': array([[1038,  952],
       

In [5]:
import numpy as np
import cv2
from PIL import Image, ImageDraw, ImageFont

# Create a white background image (adjust size as needed)
with Image.open(img_fp) as img:
    img_width, img_height = img.size

# Create a white background image with same dimensions as original
img = Image.new('RGB', (img_width, img_height), (255, 255, 255))
draw = ImageDraw.Draw(img)

# Load Chinese font
# you can download from https://huggingface.co/spaces/PaddlePaddle/PaddleOCR/blob/main/simfang.ttf
#https://github.com/PaddlePaddle/PaddleOCR/blob/main/doc/fonts/simfang.ttf
font_path = './simfang.ttf'

for item in out:
    text = item['text']
    if not text:  # Skip empty text
        continue
        
    # Get the bounding box coordinates
    box = item['position']
    x_min, y_min = box[0][0], box[0][1]
    x_max, y_max = box[2][0], box[2][1]
    
    # Calculate text width and height
    text_width = x_max - x_min
    text_height = y_max - y_min
    
    # Calculate font size based on text height
    font_size = int((text_height + text_width/len(text))*0.35)  # Adjust factor as needed

    if font_size < 10:
        font_size = 10  # Minimum font size
    if font_size > 200:
        font_size = 200
    try:
        font = ImageFont.truetype(font_path, font_size)
    except:
        font = ImageFont.load_default()
    
    # Calculate text position (centered in the box)
    text_bbox = draw.textbbox((0, 0), text, font=font)
    text_w = text_bbox[2] - text_bbox[0]
    text_h = text_bbox[3] - text_bbox[1]
    
    # Calculate position to center text in the box
    x_pos = x_min + (text_width - text_w) // 2
    y_pos = y_min + (text_height - text_h) // 2
    
    # Draw the text
    draw.text((x_pos, y_pos), text, font=font, fill=(0, 0, 0))

# Save or show the image
img.save('output_text.png')
img.show()